# 1. 라이브러리, 데이터 로드

In [30]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import pyproj
from pyproj import Proj, transform

In [3]:
df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12176\549052057.py:1: DtypeWarning: Columns (13,36,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12176\549052057.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')


In [4]:
df.keys()

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '위생업태명', '남성종사자수', '여성종사자수',
       '영업장주변구분명', '등급구분명', '급수시설구분명', '총인원', '본사종업원수', '공장사무직종업원수',
       '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모',
       '전통업소지정번호', '전통업소주된음식', '홈페이지'],
      dtype='object')

In [5]:
gg_df.keys()

Index(['시군명', '사업장명', '인허가일자', '소재지도로명주소', '소재지지번주소', '소재지우편번호', '위도', '경도',
       '인허가취소일자', '통합영업상태명', '영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자',
       '소재지시설전화번호', '소재지면적정보', '업태구분명정보', 'X좌표값', 'Y좌표값', '위생업태명', '남성종사자수',
       '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총종업원수', '본사종업원수',
       '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모정보', '전통업소지정번호', '전통업소주된음식', '홈페이지URL'],
      dtype='object')

# 2. 서울 데이터 처리

1) 서울시의 휴게음식점 인허가 정보 데이터기 때문에 사업장명에 '스타벅스'를 포함한 데이터를 가져옴
2) 그 중 필요한 컬럼 '지번주소', '사업장명', '상세영업상태명', '인허가일자', '폐업일자', '좌표정보(X)', '좌표정보(Y)'를 가져오고, 경기도와 컬럼 이름을 통일해주었음
3) 개업일자의 형태가 0000-00-00이기 때문에 부동산 데이터와 통일하기 위해 '-'를 없애고, 년도, 월 정보만 가져왔음
4) 폐업일자에 결측치가 존재함 === 폐업을 하지 않은 상태이기 때문에 미래 날짜인 2030년 1월로 통일하고, 개업일자와 마찬가지로 '-'를 제거하고 년도, 월 정보만 가져옴

5) 지번 주소를 서울/경기 구분, 시/군/구, 동으로 구분하기 위해 빈칸으로 나눠줌,
6) 통일되지 않은 형식을 맞춰주고, 각 컬럼의 값을 서울/경기, 시/군/구, 동, 번지로 설정해주었음

In [6]:
seoul_sb_df = df[df['사업장명'].str.contains('스타벅스')]
seoul_sb_df = seoul_sb_df[['지번주소','사업장명','상세영업상태명', '인허가일자', '폐업일자', '좌표정보(X)', '좌표정보(Y)']]
seoul_sb_df.reset_index(drop=True, inplace=True)
seoul_sb_df = seoul_sb_df.rename(columns={'좌표정보(X)':'X좌표값','좌표정보(Y)':'Y좌표값', '상세영업상태명' : '영업상태', '인허가일자' : '개업일자'})

seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str.replace('-','')
seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].fillna('203001')
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str.replace('-','')

In [7]:
splited = seoul_sb_df['지번주소'].str.split(' ', expand=True)
splited[3] = splited[3].replace('산','1-3')
splited[3] = splited[3].str.replace('번지','')
new_splited = splited.loc[:, 0:3]
seoul_sb_df = pd.concat([new_splited,seoul_sb_df], axis=1)
seoul_sb_df = seoul_sb_df.drop(['지번주소'], axis=1)
seoul_sb_df = seoul_sb_df.rename(columns={0 : '서울/경기', 1 : '시/군/구', 2:'동', 3:'번지'})

# 3. 경기도 데이터 처리

1) 경기도 휴게음식점 인허가 정보 데이터기 때문에 사업장명에 '스타벅스'를 포함한 데이터를 가져옴
2) 그 중 필요한 컬럼 '소재지지번주소', '사업장명', '통합영업상태명', '인허가일자', '폐업일자', 'X좌표값', 'Y좌표값'을 가져오고, 컬럼 이름 통일해주었음
3) 개업일자의 형태가 0000-00-00이기 때문에 부동산 데이터와 통일하기 위해 '-'를 없애고, 년도, 월 정보만 가져왔음
4) 폐업일자에 결측치가 존재함 === 폐업을 하지 않은 상태이기 때문에 미래 날짜인 2030년 1월로 통일하고, 개업일자와 마찬가지로 '-'를 제거하고 년도, 월 정보만 가져옴
5) 영업상태가 서울과 통일되지 않은 형태여서, 통일해주었음

5) 지번 주소를 서울/경기 구분, 시/군/구, 동으로 구분하기 위해 데이터 처리 함
6) 통일되지 않은 형식을 맞춰주고, 각 컬럼의 값을 서울/경기, 시/군/구, 동, 리, 번지로 설정해주었음

In [8]:
seoul_sb_df.head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,203001,193414.6768,450675.0036
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,203001,201279.4432,442806.7092
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,203001,200800.4170,444989.8743
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,203001,204537.4607,444036.1667
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,203001,205464.7204,443592.9735


In [9]:
gg_sb_df = gg_df[gg_df['사업장명'].str.contains('스타벅스')]
gg_sb_df = gg_sb_df[['소재지지번주소','사업장명','통합영업상태명', '인허가일자', '폐업일자', 'X좌표값', 'Y좌표값','위도','경도']]
gg_sb_df.reset_index(drop=True, inplace=True)
gg_sb_df = gg_sb_df.rename(columns={'통합영업상태명' : '영업상태', '인허가일자' : '개업일자','소재지지번주소':'지번주소'})

gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str.replace('-','')
gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].fillna('203001')
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str.replace('-','')
gg_sb_df['영업상태'] = gg_sb_df['영업상태'].str.replace('영업/정상','영업')

In [10]:
gg_splited = gg_sb_df['지번주소'].str.split(' ', expand=True)
gg_splited.loc[gg_splited[2].str[-1] == '구', 1] = gg_splited.loc[gg_splited[2].str[-1] == '구', 1] + gg_splited.loc[gg_splited[2].str[-1] == '구', 2]
gg_splited.loc[gg_splited[2].str[-1] == '구', 2] = gg_splited.loc[gg_splited[2].str[-1] == '구', 3]
gg_splited.loc[gg_splited[1].str[-1] == '구', 3] = ''

In [11]:
gg_splited_n = gg_splited.copy()
mask = gg_splited_n[3].str[-1].str.isdigit() | (gg_splited_n[3].str[-2:] == '번지')
gg_splited_n[4] = np.where(mask, gg_splited_n[3], gg_splited_n[4])
gg_splited_n.loc[gg_splited_n[3] == '', 4] = gg_splited[4]
gg_splited_n[3] = np.where(mask, '', gg_splited_n[3])
gg_splited_n[4] = gg_splited_n[4].str.replace('번지','')

In [12]:
gg_splited_n[4].unique()

array(['113-4', '768', '699', '0', '332-1', '999', '529', '814', '890-5',
       '1296-3', '370', '373-9', '1050-185', '469', '142-84', '1640',
       '879-2', '1836-3', '81', '18-2', '2602', '973', '689-19', '2700',
       '2222-3', '770-1', '17', '291', '1755', '630', '868', '855', '609',
       '106-3', '568', '628-2', '1332', '620-1', '71-1', '1084-1', '2703',
       '1809', '762', '1237', '751', '1531', '970-5', '1-19', '40-10',
       '79', '425', '500', '158-86', '1077', '1335', '512-3', '513',
       '482', '35', '53-3', '245-9', '12-12', '701-5', '168', '20-14',
       '261-3', '399-6', '526-3', '854-2', '344-6', '874-5', '567-1',
       '383-101', '596', '916', '111', '1145-6', '1132-3', '1099-1',
       '1142-5', '264-4', '774-4', '646-1', '917', '2083-2', '1298-1',
       '2031', '654', '57-1', '326-3', '502-20', '6886-6', '246-6',
       '6880-9', '1608', '620-11', '62-25', '146-26', '59-3', '230-6',
       '97-3', '6147-2', '4001-29', '6216-4', '6058-1', '806-1',
       '

In [13]:
gg_new_splited = gg_splited_n.loc[:, 0:4]
gg_sb_df = pd.concat([gg_new_splited,gg_sb_df], axis=1)
gg_sb_df = gg_sb_df.drop(['지번주소'], axis=1)

gg_sb_df[1] = gg_sb_df[1].str.replace('고양시','고양')
gg_sb_df[1] = gg_sb_df[1].str.replace('성남시','성남')
gg_sb_df[1] = gg_sb_df[1].str.replace('수원시','수원')
gg_sb_df[1] = gg_sb_df[1].str.replace('안산시','안산')
gg_sb_df[1] = gg_sb_df[1].str.replace('안양시','안양')
gg_sb_df[1] = gg_sb_df[1].str.replace('용인시','용인')

gg_sb_df = gg_sb_df.rename(columns={0 : '서울/경기', 1 : '시/군/구', 2:'동', 3:'리', 4:'번지'})
gg_sb_df['동'] = gg_sb_df['동'].str.replace('매산로1가','매산동')
gg_sb_df['번지'].unique()

array(['113-4', '768', '699', '0', '332-1', '999', '529', '814', '890-5',
       '1296-3', '370', '373-9', '1050-185', '469', '142-84', '1640',
       '879-2', '1836-3', '81', '18-2', '2602', '973', '689-19', '2700',
       '2222-3', '770-1', '17', '291', '1755', '630', '868', '855', '609',
       '106-3', '568', '628-2', '1332', '620-1', '71-1', '1084-1', '2703',
       '1809', '762', '1237', '751', '1531', '970-5', '1-19', '40-10',
       '79', '425', '500', '158-86', '1077', '1335', '512-3', '513',
       '482', '35', '53-3', '245-9', '12-12', '701-5', '168', '20-14',
       '261-3', '399-6', '526-3', '854-2', '344-6', '874-5', '567-1',
       '383-101', '596', '916', '111', '1145-6', '1132-3', '1099-1',
       '1142-5', '264-4', '774-4', '646-1', '917', '2083-2', '1298-1',
       '2031', '654', '57-1', '326-3', '502-20', '6886-6', '246-6',
       '6880-9', '1608', '620-11', '62-25', '146-26', '59-3', '230-6',
       '97-3', '6147-2', '4001-29', '6216-4', '6058-1', '806-1',
       '

In [14]:
seoul_sb_df.head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,203001,193414.6768,450675.0036
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,203001,201279.4432,442806.7092
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,203001,200800.4170,444989.8743
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,203001,204537.4607,444036.1667
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,203001,205464.7204,443592.9735


In [15]:
gg_sb_df.head()

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,경기도,가평군,가평읍,달전리,113-4,스타벅스남이섬점,영업,202107,203001,246197.235727,478362.818093,37.806577,127.525344
1,경기도,고양일산동구,장항동,,768,(주)스타벅스커피코리아일산라페스타점,폐업,200308,201306,179538.773119,462032.199846,37.660351,126.768854
2,경기도,고양일산동구,풍동,,699,스타벅스 고양대로DT점,영업,202206,203001,181927.758553,463930.090165,37.677671,126.796117
3,경기도,고양덕양구,덕은동,,0,스타벅스 고양덕은점,영업,202303,203001,NaN,NaN,37.579442,126.870657
4,경기도,고양덕양구,원흥동,,332-1,스타벅스 고양원흥DT점,영업,202108,203001,188391.022888,459392.615118,37.636805,126.869312


In [16]:
# seoul_sb_df.to_csv('./seoul_sb_처리중_2.csv', index=False)
# gg_sb_df.to_csv('./gg_sb_처리중_2.csv', index=False)

# 3. X좌표, Y좌표와 위도, 경도 값 처리

1. 서울
  - 서울은 경기도와 다르게 위도, 경도 값이 존재하지 않아서 가진 주소를 가지고 X좌표, Y좌표의 결측치를 채웠음
  - X좌표와 Y좌표를 위도, 경도로 변환했음


2. 경기도
  - 경기도는 위도, 경도 값이 있어서 이를 이용해 X좌표, Y좌표의 결측치를 채웠음

In [17]:
gg_1 = pd.read_csv('./gg_sb_처리중_2.csv')
seoul_1 = pd.read_csv('./seoul_sb_처리중_2.csv')

In [18]:
seoul_1.loc[seoul_1['X좌표값'].isnull()].head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
90,서울특별시,마포구,신공덕동,69-2,스타벅스마포이마트,영업,201201,203001,NaN,NaN
131,서울특별시,송파구,가락동,479,스타벅스 송파헬리오시티점,영업,202004,203001,NaN,NaN
136,서울특별시,용산구,한강로2가,342,스타벅스신용산역점,영업,201708,203001,NaN,NaN
170,서울특별시,중구,남대문로5가,253,스타벅스 숭례문에스지타워,영업,202008,203001,NaN,NaN
171,서울특별시,중구,수표동,47-1,스타벅스 청계크리스탈스퀘어점,영업,202203,203001,NaN,NaN


In [19]:
gg_1.loc[gg_1['X좌표값'].isnull()].head()

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
3,경기도,고양덕양구,덕은동,NaN,0,스타벅스 고양덕은점,영업,202303,203001,NaN,NaN,37.579442,126.870657
23,경기도,고양덕양구,지축동,NaN,0,스타벅스 지축역점,영업,202212,203001,NaN,NaN,37.649763,126.914286
68,경기도,광주시,신현동,NaN,701-5,스타벅스 경기광주신현DT점,영업,201812,203001,NaN,NaN,37.359865,127.153177
77,경기도,구리시,토평동,NaN,567-1,스타벅스 구리토평DT,영업,202112,203001,NaN,NaN,37.584568,127.147300
86,경기도,김포시,걸포동,NaN,264-4,스타벅스 김포걸포점,영업,202203,203001,NaN,NaN,37.633144,126.701030


In [ ]:
seoul_1

### 서울

In [20]:
# from geopy.geocoders import Nominatim
# import time

geolocator = Nominatim(user_agent="aa")

def get_geocode(address):
    location = geolocator.geocode(address)
    if location is None:
        return None, None
    else:
        return location.latitude, location.longitude
    
for i, row in seoul_1.loc[seoul_1['X좌표값'].isna() | seoul_1['Y좌표값'].isna()].iterrows():
    address = row['시/군/구'] + ' ' + row['동'] + ' ' + row['번지']
    lat, lon = get_geocode(address)
    seoul_1.at[i, 'X좌표값'] = lon
    seoul_1.at[i, 'Y좌표값'] = lat
    time.sleep(1) 

In [21]:
# import pyproj
# from pyproj import Proj, transform

in_proj = Proj(init='epsg:5186')
out_proj = Proj(init='epsg:4326')

seoul_1['위도'] = 0
seoul_1['경도'] = 0

for i in range(len(seoul_1)):
    x = seoul_1.at[i, 'X좌표값']
    y = seoul_1.at[i, 'Y좌표값']
    lon, lat = transform(in_proj, out_proj, x, y)
    seoul_1.at[i, '위도'] = lat
    seoul_1.at[i, '경도'] = lon

c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12176\1138771573.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotch

In [22]:
seoul_1.isnull().sum()

서울/경기    0
시/군/구    0
동        0
번지       0
사업장명     0
영업상태     0
개업일자     0
폐업일자     0
X좌표값     0
Y좌표값     0
위도       0
경도       0
dtype: int64

In [23]:
seoul_1.head()

,서울/경기,시/군/구,동,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,서울특별시,마포구,동교동,173-14,스타벅스 홍대동교점,영업,202303,203001,193414.6768,450675.0036,36.654511,126.926349
1,서울특별시,서초구,서초동,1623-1,스타벅스 서울교대점,영업,201007,203001,201279.4432,442806.7092,36.583628,127.014296
2,서울특별시,서초구,반포동,20-45,스타벅스 반포자이점,영업,202201,203001,200800.4170,444989.8743,36.603302,127.008946
3,서울특별시,강남구,대치동,922-1,스타벅스 대치한티점,영업,202101,203001,204537.4607,444036.1667,36.594697,127.050708
4,서울특별시,강남구,대치동,599,스타벅스 대치재경빌딩B1,영업,202012,203001,205464.7204,443592.9735,36.590699,127.061068


### 경기도

In [24]:
mask = (gg_1['X좌표값'].isnull()) & (gg_1['Y좌표값'].isnull())
filtered_df = gg_1[mask]

proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init='epsg:4326')

for index, row in filtered_df.iterrows():
    lon, lat = row['경도'], row['위도']
    x, y = transform(proj_WGS84, proj_UTMK, lon, lat)
    gg_1.loc[index, 'X좌표값'] = x
    gg_1.loc[index, 'Y좌표값'] = y

c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12176\807109029.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas

In [25]:
gg_1.isnull().sum()

서울/경기      0
시/군/구      0
동          0
리        422
번지         0
사업장명       0
영업상태       0
개업일자       0
폐업일자       0
X좌표값       0
Y좌표값       0
위도         0
경도         0
dtype: int64

In [26]:
gg_1.head()

,서울/경기,시/군/구,동,리,번지,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값,위도,경도
0,경기도,가평군,가평읍,달전리,113-4,스타벅스남이섬점,영업,202107,203001,246197.235727,4.783628e+05,37.806577,127.525344
1,경기도,고양일산동구,장항동,NaN,768,(주)스타벅스커피코리아일산라페스타점,폐업,200308,201306,179538.773119,4.620322e+05,37.660351,126.768854
2,경기도,고양일산동구,풍동,NaN,699,스타벅스 고양대로DT점,영업,202206,203001,181927.758553,4.639301e+05,37.677671,126.796117
3,경기도,고양덕양구,덕은동,NaN,0,스타벅스 고양덕은점,영업,202303,203001,944620.173415,1.953220e+06,37.579442,126.870657
4,경기도,고양덕양구,원흥동,NaN,332-1,스타벅스 고양원흥DT점,영업,202108,203001,188391.022888,4.593926e+05,37.636805,126.869312


In [27]:
# seoul_1.to_csv('./seoul_sb_처리중_3.csv', index=False)
# gg_1.to_csv('./gg_sb_처리중_3.csv', index=False)

# 4. 경기도 '리' 결측치 어떻게 채울지 고민

In [28]:
# gg_1['리'].fillna('', inplace=True)

# 5. 서울, 경기도 어디가 가장 가까움? (미완성, 할지말지)

In [36]:
seoul_sb = pd.read_csv('./seoul_sb_처리중_3.csv')
gg_sb = pd.read_csv('./gg_sb_처리중_3.csv')

In [43]:
seoul_ = pd.read_csv('./서울_한글.csv')
gg_ = pd.read_csv('./경기도_한글.csv')

In [45]:
seoul_2 = pd.read_csv('./서울_숫자.csv')

In [54]:
gg_2 = pd.read_csv('./경기도_숫자.csv')

In [ ]:
from math import sin, cos, sqrt, atan2, radians


def calc_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

seoul_lat = seoul_['위도']
seoul_lon = seoul_['경도']
starbucks_lat = starbucks['위도']
starbucks_lon = starbucks['경도']

distances = []
for i in range(len(seoul)):
    row_lat, row_lon = seoul_lat[i], seoul_lon[i]
    row_distances = []
    for j in range(len(starbucks)):
        sb_lat, sb_lon = starbucks_lat[j], starbucks_lon[j]
        dist = calc_distance(row_lat, row_lon, sb_lat, sb_lon)
        row_distances.append(dist)
    distances.append(row_distances)

min_idx = np.argmin(distances, axis=1)

seoul['최소'] = np.min(distances, axis=1)

# 6. 스타벅스 동 단위로 몇개있는지 카운팅

In [94]:
seoul_sb = pd.read_csv('./seoul_sb_처리중_3.csv')
gg_sb = pd.read_csv('./gg_sb_처리중_3.csv')

### 1) 서울

In [99]:
address = seoul_sb['시/군/구'] + ' ' + seoul_sb['동']

count_by_address = address.value_counts()

df = pd.DataFrame({'시군구동': count_by_address.index, '스타벅스 개수': count_by_address.values})
df

,시군구동,스타벅스 개수
0,강남구 역삼동,36
1,서초구 서초동,25
2,영등포구 여의도동,25
3,강남구 삼성동,24
4,강남구 대치동,18
...,...,...
238,용산구 효창동,1
239,용산구 청파동3가,1
240,용산구 동빙고동,1
241,용산구 용산동5가,1


### 2) 경기도

In [100]:
address_2 = gg_sb['시/군/구'] + ' ' + gg_sb['동']

count_by_address_2 = address_2.value_counts()

df_2 = pd.DataFrame({'시군구동': count_by_address_2.index, '스타벅스 개수': count_by_address_2.values})
df_2

,시군구동,스타벅스 개수
0,부천시 중동,11
1,성남분당구 정자동,9
2,성남분당구 삼평동,8
3,성남분당구 서현동,8
4,고양일산동구 장항동,7
...,...,...
226,수원장안구 송죽동,1
227,수원장안구 천천동,1
228,수원장안구 파장동,1
229,수원영통구 신동,1


In [113]:
seoul_ = pd.read_csv('./서울_한글.csv')
gg_ = pd.read_csv('./경기도_한글.csv')

In [114]:
seoul_['시군구동'] = seoul_['시/군/구'] + ' ' + seoul_['동']
gg_['시군구동'] = gg_['시/군/구'] + ' ' + gg_['동']


In [115]:
seoul_ = pd.merge(seoul_, df, left_on='시군구동', right_on='시군구동').drop(['시군구동'], axis=1)

In [116]:
gg_ = pd.merge(gg_, df_2, left_on ='시군구동', right_on = '시군구동').drop(['시군구동'], axis=1)

In [121]:
seoul_.sample(n=5)

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder,동_Encoder,스타벅스 개수
1406194,동대문구,답십리동,1,59.980,5000,137,7.0,0,0.50,2108.33,2020.0,1.0,10,58,1
1356192,도봉구,방학동,1,74.910,4000,40,3.0,2,1.50,1961.31,2015.0,29.0,9,134,1
1759532,서대문구,남가좌동,0,59.981,61000,0,5.0,0,1.25,2663.34,2022.0,7.0,13,41,3
1021888,구로구,구로동,0,59.120,17000,0,6.0,1,2.75,1997.05,2012.0,7.0,6,28,9
1828606,서초구,반포동,1,84.998,5000,350,13.0,0,3.00,1905.12,2012.0,3.0,14,131,14


In [122]:
gg_.sample(n=5)

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder,동_Encoder,스타벅스 개수
2081554,용인처인구,김량장동,1,67.200,3500,35,2.0,2,2.50,1994.96,2014.0,35.0,33,95,2
1913166,용인기흥구,신갈동,1,84.504,18900,68,17.0,0,3.00,2293.61,2022.0,9.0,31,331,1
219041,고양일산서구,대화동,0,134.010,22000,0,2.0,2,2.50,2001.05,2013.0,18.0,3,141,6
634310,김포시,장기동,1,37.840,3960,17,1.0,2,0.50,3061.42,2021.0,0.0,9,466,3
37893,고양덕양구,주교동,1,37.080,700,27,-1.0,2,1.25,2394.47,2017.0,15.0,1,491,1


In [123]:
# seoul_.to_csv('./seoul_sb_처리중_4.csv', index=False)
# gg_.to_csv('./gg_sb_처리중_4.csv', index=False)